In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv


# 1. Read the data 

In [2]:
import pandas
df_train = pandas.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test = pandas.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
pandas.set_option('display.max_rows', 500)
pandas.set_option('display.max_columns', 500)
pandas.set_option('display.width', 1000)

# 2. Extract features

In [4]:
def extract_features(df):
    """
    Input: df, input data frame
    Output: df_features, features for machine learning
    """
    
    numerical_columns = [
        column_name
        for column_name in df.columns
        if df.dtypes[column_name] in (np.int64, np.float64)
        if column_name != 'SalePrice'
    ] # list comprehension

    df_features = df.loc[:, numerical_columns].fillna(0)
    
    df_features['MSZoning'] = 0
    for value_index, value in enumerate(('RL', 'RM', 'FV', 'RH', 'C (all)')):
        df_features.loc[df['MSZoning'] == value, 'MSZoning'] = value_index 
        
    # TODO: add new features
    
    return df_features

In [5]:
X_train = extract_features(df_train)
X_test = extract_features(df_test)

y_train = df_train['SalePrice']

# 3. Train machine learning models

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
#from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=123)
model.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=123, verbose=0,
                      warm_start=False)

# 4. Make predictions

In [8]:
y_predicted = model.predict(X_test)
df_submission = pandas.DataFrame({'Id': X_test['Id'], 'SalePrice': y_predicted})
df_submission.to_csv('submission.csv', index=False)

# 5. Estimate feature importances

In [9]:
model = RandomForestRegressor(random_state=123)
model.fit(X_train, y_train)

df_importance = pandas.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_,
}).sort_values('importance', ascending=False)

df_importance

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,feature,importance
4,OverallQual,0.565998
16,GrLivArea,0.124326
26,GarageCars,0.045104
14,2ndFlrSF,0.042777
12,TotalBsmtSF,0.036402
9,BsmtFinSF1,0.030882
13,1stFlrSF,0.023847
3,LotArea,0.016453
27,GarageArea,0.013879
6,YearBuilt,0.011401
